# JSON file

In [4]:
#LEER CON LIBRERÍA: Sí tengo diccionarios no toma todos los datos.
import pandas as pd
pd.read_json('nested_json.json') 

,school_name,class,students
0,ABC primary school,Year 1,"{'id': 'A001', 'name': 'Tom', 'math': 60, 'phy..."
1,ABC primary school,Year 1,"{'id': 'A002', 'name': 'James', 'math': 89, 'p..."
2,ABC primary school,Year 1,"{'id': 'A003', 'name': 'Jenny', 'math': 79, 'p..."


In [6]:
#LEER CON LIBRERÍA JSON
import json
# Normalizar los datos
with open('nested_json.json','r') as f:
    data = json.loads(f.read())# Flatten data
data

{'school_name': 'ABC primary school',
 'class': 'Year 1',
 'students': [{'id': 'A001',
   'name': 'Tom',
   'math': 60,
   'physics': 66,
   'chemistry': 61},
  {'id': 'A002', 'name': 'James', 'math': 89, 'physics': 76, 'chemistry': 51},
  {'id': 'A003', 'name': 'Jenny', 'math': 79, 'physics': 90, 'chemistry': 78}]}

In [7]:
type(data)

dict

In [8]:
data['students']

[{'id': 'A001', 'name': 'Tom', 'math': 60, 'physics': 66, 'chemistry': 61},
 {'id': 'A002', 'name': 'James', 'math': 89, 'physics': 76, 'chemistry': 51},
 {'id': 'A003', 'name': 'Jenny', 'math': 79, 'physics': 90, 'chemistry': 78}]

In [9]:
pd.json_normalize?

Signature:
pd.json_normalize(
    data: 'dict | list[dict]',
    record_path: 'str | list | None' = None,
    meta: 'str | list[str | list[str]] | None' = None,
    meta_prefix: 'str | None' = None,
    record_prefix: 'str | None' = None,
    errors: 'str' = 'raise',
    sep: 'str' = '.',
    max_level: 'int | None' = None,
) -> 'DataFrame'
Docstring:
Normalize semi-structured JSON data into a flat table.

Parameters
----------
data : dict or list of dicts
    Unserialized JSON objects.
record_path : str or list of str, default None
    Path in each object to list of records. If not passed, data will be
    assumed to be an array of records.
meta : list of paths (str or list of str), default None
    Fields to use as metadata for each record in resulting table.
meta_prefix : str, default None
    If True, prefix records with dotted (?) path, e.g. foo.bar.field if
    meta is ['foo', 'bar'].
record_prefix : str, default None
    If True, prefix records with dotted (?) path, e.g. foo.bar

In [10]:
df_nested_list = pd.json_normalize(data, record_path =['students']) #Solo toma datos de esa columna
df_nested_list

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


In [11]:
# Combinando
df_nested_list = pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['school_name', 'class'] #Activo columna de contexto
)
df_nested_list

,id,name,math,physics,chemistry,school_name,class
0,A001,Tom,60,66,61,ABC primary school,Year 1
1,A002,James,89,76,51,ABC primary school,Year 1
2,A003,Jenny,79,90,78,ABC primary school,Year 1


# SQL

In [17]:
#PARA PASAR DIRECTAMENTE LA QUERY SQL
pd.read_sql_query?

#Necesita SQL y conexión

Signature:
pd.read_sql_query(
    sql,
    con,
    index_col=None,
    coerce_float: 'bool' = True,
    params=None,
    parse_dates=None,
    chunksize: 'int | None' = None,
    dtype: 'DtypeArg | None' = None,
) -> 'DataFrame | Iterator[DataFrame]'
Docstring:
Read SQL query into a DataFrame.

Returns a DataFrame corresponding to the result set of the query
string. Optionally provide an `index_col` parameter to use one of the
columns as the index, otherwise default integer index will be used.

Parameters
----------
sql : str SQL query or SQLAlchemy Selectable (select or text object)
    SQL query to be executed.
con : SQLAlchemy connectable, str, or sqlite3 connection
    Using SQLAlchemy makes it possible to use any DB supported by that
    library. If a DBAPI2 object, only sqlite3 is supported.
index_col : str or list of str, optional, default: None
    Column(s) to set as index(MultiIndex).
coerce_float : bool, default True
    Attempts to convert values of non-string, non-numeric

In [15]:
pip install matplotlib

                                              0.0/7.6 MB ? eta -:--:--
                                              0.1/7.6 MB 1.7 MB/s eta 0:00:05
                                              0.2/7.6 MB 2.1 MB/s eta 0:00:04
     -                                        0.2/7.6 MB 2.1 MB/s eta 0:00:04
     -                                        0.4/7.6 MB 2.1 MB/s eta 0:00:04
     --                                       0.5/7.6 MB 2.1 MB/s eta 0:00:04
     --                                       0.5/7.6 MB 2.0 MB/s eta 0:00:04
     ---                                      0.6/7.6 MB 2.0 MB/s eta 0:00:04
     ---                                      0.7/7.6 MB 1.9 MB/s eta 0:00:04
     ---                                      0.7/7.6 MB 1.8 MB/s eta 0:00:04
     ----                                     0.9/7.6 MB 2.0 MB/s eta 0:00:04
     -----                                    1.0/7.6 MB 1.9 MB/s eta 0:00:04
     -----                                    1.1/7.6 MB 1.9 MB/s eta 0

In [19]:
#Tomo info de base de datos local sqlite (Por lo cual no necesito Puerto, pwd, user y host)
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
conn= sql.connect('nba_salary.sqlite')
conn

In [20]:
sql.connect?

Docstring:
connect(database[, timeout, detect_types, isolation_level,
        check_same_thread, factory, cached_statements, uri])

Opens a connection to the SQLite database file *database*. You can use
":memory:" to open a database connection to a database that resides in
RAM instead of on disk.
Type:      builtin_function_or_method

In [21]:
# Visualizar las tablas
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

#Devuelve una tupla con cada registro

[('NBA_season1718_salary',), ('Seasons_Stats',)]


In [23]:
#Función cita el SQL y la conexión
df_salary = pd.read_sql_query("SELECT * from NBA_season1718_salary", conn)
df_salary.head()

,X1,Player,Tm,season17_18
0,1.0,Stephen Curry,GSW,34682550.0
1,2.0,LeBron James,CLE,33285709.0
2,3.0,Paul Millsap,DEN,31269231.0
3,4.0,Gordon Hayward,BOS,29727900.0
4,5.0,Blake Griffin,DET,29512900.0


In [24]:
df_stats = pd.read_sql_query("SELECT * from Seasons_Stats", conn)
df_stats.head()

,X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0.0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1.0,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2.0,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3.0,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4.0,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [26]:
df_stats.columns

Index(['X1', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [27]:
# Ordenar jugadores por mayor cantidad de puntos totales
df_stats.sort_values(by=['PTS','ORB'],ascending=False)

,X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1706,1706.0,1962.0,Wilt Chamberlain*,C,25.0,PHW,80.0,NaN,3882.0,31.7,...,0.613,NaN,NaN,2052.0,192.0,NaN,NaN,NaN,123.0,4029.0
1827,1827.0,1963.0,Wilt Chamberlain*,C,26.0,SFW,80.0,NaN,3806.0,31.8,...,0.593,NaN,NaN,1946.0,275.0,NaN,NaN,NaN,136.0,3586.0
8469,8469.0,1987.0,Michael Jordan*,SG,23.0,CHI,82.0,NaN,3281.0,29.8,...,0.857,NaN,NaN,430.0,377.0,NaN,NaN,NaN,237.0,3041.0
1593,1593.0,1961.0,Wilt Chamberlain*,C,24.0,PHW,79.0,NaN,3773.0,27.8,...,0.504,NaN,NaN,2149.0,148.0,NaN,NaN,NaN,130.0,3033.0
1962,1962.0,1964.0,Wilt Chamberlain*,C,27.0,SFW,80.0,NaN,3689.0,31.6,...,0.531,NaN,NaN,1787.0,403.0,NaN,NaN,NaN,182.0,2948.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21678,21678.0,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22252,22252.0,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22864,22864.0,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23516,23516.0,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
